In [ ]:
import math
import numpy as np 
from numpy.linalg import inv
import matplotlib.pyplot as plt
import openpyxl
import cmath
import graphviz
from array import array

import qiskit as q
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.visualization import *
from qiskit.quantum_info import Pauli, SparsePauliOp, Operator
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import *
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_aer.primitives import EstimatorV2 as AerEstimator
from qiskit_aer.primitives import SamplerV2 as AerSampler
from qiskit_ibm_runtime import QiskitRuntimeService

In [5]:
# Save an IBM Quantum account and set it as your default account.
API_Token = '13d9540e280b61b7a4254fcdb05516180957df07523dfef1924f7d9363699a51826b42e7a063b3fb685f295f6d27f8ff61810a97f503a704c07bc959d6cc6e3f'
token = "PJubuPXreZKAxan0yV6OLEIkdKipFYGgvvkGQSZAtpMU"
instance = 'crn:v1:bluemix:public:quantum-computing:us-east:a/f071734952cb4c9993a642d0a87d18bb:392306ae-fe55-475c-a8c5-978d0ba1871e::'
QiskitRuntimeService.save_account(
    channel="ibm_quantum_platform",
    token=token,
    instance=instance,
    set_as_default=True,
    overwrite=True
    )

In [7]:
# Load saved credentials
service = QiskitRuntimeService()
#For real Device
#backend = service.least_busy(operational=True, simulator=False)
backend_name = "ibm_fez"
device = service.backend(backend_name)
noise_model = NoiseModel.from_backend(device)
basis_gates = noise_model.basis_gates
coupling_map = device.configuration().coupling_map

#For Aer
backend = AerSimulator(noise_model=noise_model,
                       coupling_map=coupling_map,
                       basis_gates=basis_gates)
#backend = AerSimulator()
#backend = AerSimulator.from_backend(backend_nazca)

backendqubitNum = backend.num_qubits

In [8]:
# 1) 建立最小電路：H(0) -> CNOT(0,1)
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

In [10]:
q_meas = {}

# Function to generate measurement circuits
def generate_meas_circuits(prefix, index):
    circuits = {
        f"{prefix}measI": QuantumCircuit(qregs, creg),
        f"{prefix}measX": QuantumCircuit(qregs, creg),
        f"{prefix}measY": QuantumCircuit(qregs, creg),
        f"{prefix}measZ": QuantumCircuit(qregs, creg),
    }

    # X-measurement
    circuits[f"{prefix}measX"].h(index)
    circuits[f"{prefix}measX"].measure(index, index)

    # Y-measurement
    circuits[f"{prefix}measY"].sdg(index)
    circuits[f"{prefix}measY"].h(index)
    circuits[f"{prefix}measY"].measure(index, index)

    # Z-measurement
    circuits[f"{prefix}measZ"].measure(index, index)

    return circuits

# Generate circuits for q0 and q1
q_meas.update(generate_meas_circuits("q0", 0))
q_meas.update(generate_meas_circuits("q1", 1))

# Store the measurement circuits in a dictionary
Measurement = {}
for q0_label, q0_meas in q_meas.items():
    for q1_label, q1_meas in q_meas.items():
        meas_comb = q.circuit.QuantumCircuit.compose(q0_meas, q1_meas)
        Measurement.update({f"{q0_label}_{q1_label}": meas_comb})

In [11]:
#Build each set of circuits with four kinds of Initialstates and four kinds of measurements
Circuit = {}
for MeasName, Measure in Measurement.items():
    for InitName, Initial in InitialState.items():
        circuit = q.circuit.QuantumCircuit.compose(Initial, Measure)
        Circuit.update({f'{InitName}_{MeasName}':circuit})

In [12]:
len(Circuit)

256

In [13]:
shots = 1000
sampler = Sampler(mode=backend)

job = {}
for Name, Cir in Circuit.items():
    CirTran = q.compiler.transpile(Cir, backend=backend, optimization_level=0)
    job.update({Name:sampler.run([CirTran], shots=shots)})

In [14]:
result_origin = []
for index, job_name in job.items():
    res = job_name.result()
    result_origin.append(res[0].data.c.get_counts())
result_origin

[{'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'10': 508, '00': 492},
 {'10': 492, '00': 508},
 {'00': 1000},
 {'00': 503, '10': 497},
 {'10': 482, '00': 518},
 {'00': 517, '10': 483},
 {'00': 995, '10': 5},
 {'00': 496, '10': 504},
 {'10': 494, '00': 506},
 {'00': 510, '10': 490},
 {'00': 999, '10': 1},
 {'00': 514, '10': 486},
 {'00': 465, '10': 535},
 {'10': 511, '00': 489},
 {'00': 997, '10': 3},
 {'10': 538, '00': 462},
 {'00': 504, '10': 496},
 {'10': 517, '00': 483},
 {'00': 495, '10': 505},
 {'00': 999, '10': 1},
 {'10': 507, '00': 493},
 {'10': 498, '00': 502},
 {'00': 501, '10': 499},
 {'00': 996, '10': 4},
 {'10': 525, '00': 475},
 {'10': 505, '00': 495},
 {'00': 478, '10': 522},
 {'00': 999, '10': 1},
 {'00': 473, '10': 527},
 {'10': 488, '00': 512},
 {'10': 485, '00': 515},
 {'00':

In [15]:
# Required keys for each dictionary
required_keys = ['00', '01', '10', '11']

# Function to fill missing keys with 0
def fill_missing_keys(dicts, keys):
    return [{k: d.get(k, 0) for k in keys} for d in dicts]

# Fill missing keys in the input data
result = fill_missing_keys(result_origin, required_keys)
result

[{'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 492, '01': 0, '10': 508, '11': 0},
 {'00': 508, '01': 0, '10': 492, '11': 0},
 {'00': 1000, '01': 0, '10': 0, '11': 0},
 {'00': 503, '01': 0, '10': 497, '11': 0},
 {'00': 518, '01': 0, '10': 482, '11': 0},
 {'00': 517, '01': 0, '10': 483, '11': 0},
 {'00': 995, '01': 0, '10': 5, '11': 0},
 {'00': 496, '01': 0, '10': 50

In [38]:
#Create GramMatrix g
g = np.ones((16,16))
for i in range (16, len(result)):
    row = int(i/16)
    column = i%16
    if row <= 3:
        g[row][column] = (result[i]['00'] - result[i]['10'] + result[i]['01'] - result[i]['01'])/1000
    elif row==4 or row==8 or row==12:
        g[row][column] = (result[i]['00'] + result[i]['10'] - result[i]['01'] - result[i]['01'])/1000
    else:
        g[row][column] = (result[i]['00'] - result[i]['10'] - result[i]['01'] + result[i]['01'])/1000
g

array([[ 1.000e+00,  1.000e+00,  1.000e+00,  1.000e+00,  1.000e+00,
         1.000e+00,  1.000e+00,  1.000e+00,  1.000e+00,  1.000e+00,
         1.000e+00,  1.000e+00,  1.000e+00,  1.000e+00,  1.000e+00,
         1.000e+00],
       [-1.600e-02,  1.600e-02,  1.000e+00,  6.000e-03,  3.600e-02,
         3.400e-02,  9.900e-01, -8.000e-03,  1.200e-02,  2.000e-02,
         9.980e-01,  2.800e-02, -7.000e-02, -2.200e-02,  9.940e-01,
        -7.600e-02],
       [ 8.000e-03, -3.400e-02, -1.000e-02,  9.980e-01, -1.400e-02,
         4.000e-03,  2.000e-03,  9.920e-01, -5.000e-02, -1.000e-02,
        -4.400e-02,  9.980e-01, -5.400e-02,  2.400e-02,  3.000e-02,
         1.000e+00],
       [ 9.940e-01, -9.940e-01,  3.600e-02,  1.400e-02,  9.980e-01,
        -9.960e-01, -6.000e-02, -5.600e-02,  9.880e-01, -9.920e-01,
        -3.600e-02, -1.600e-02,  9.980e-01, -9.880e-01,  2.000e-03,
        -2.600e-02],
       [-4.910e-01, -4.520e-01, -5.390e-01, -5.480e-01, -4.280e-01,
        -4.970e-01, -5.750e-01, 

In [40]:
#Create State Preparation Matrix A
A_single = np.array([[1, 1, 1, 1],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1],
                     [1,-1, 0, 0]])
A = np.kron(A_single, A_single)
#print(A)

#Calculate Readout Matrix by the quation B = g * A^-1
A_inv = inv(A)
#print(A_inv)
B = np.matmul(g, A_inv)
B_inv = inv(B)
print(B_inv)
'''
#Calculate observable X, Y, Z
a_x = np.array([[0, 1, 0, 0]])
a_y = np.array([[0, 0, 1, 0]])
a_z = np.array([[0, 0, 0, 1]])

B_inv = inv(B)
q_x = np.matmul(a_x, B_inv)
q_y = np.matmul(a_y, B_inv)
q_z = np.matmul(a_z, B_inv)
print(B_inv, '\n\n', q_x, '\n\n', q_y, '\n\n', q_z)
'''

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.56300790e-02  1.12785034e+00  3.45325430e-02 -2.26071078e-02
  -1.50638317e-03 -6.82951425e-03 -3.53583800e-02 -7.54441507e-05
   4.68485040e-02 -1.58551578e-01  4.23185324e-02  4.51226640e-02
   1.34100030e-02 -4.14598726e-02 -5.62780294e-02  1.13856016e-02]
 [ 2.09865186e-02  2.43040674e-02  1.02146978e+00 -6.93936508e-02
   1.34863321e-02  2.89855847e-02 -3.67116091e-02  6.00575839e-02
   5.15991787e-03 -9.00650151e-02 -1.05324210e-02  9.76704970e-02
   3.00024041e-03  8.61360628e-03 -1.23589018e-02 -3.31789055e-02]
 [ 9.38674581e-04  4.54215724e-02  4.76811026e-02  9.89405525e-01
  -1.17704655e-03  5.54269120e-02 -9.20076805e-03  1.60865857e-02
  -4.73324927e-03 -2.37738189e-02  2.65798949e-02  1.26596038e-02
   2.01

0.0

In [41]:
import numpy as np
import csv

# 假設 B_inv 是你的 16x16 矩陣
# 例如：B_inv = np.array([[1, 2, 3, ...], [...], ...]) 

# 將矩陣存成CSV檔案
filename = 'matrix_B_inv.csv'
np.savetxt(filename, B_inv, delimiter=",", fmt="%.10e")

print(f"矩陣已成功存成CSV檔案：{filename}")

矩陣已成功存成CSV檔案：matrix_B_inv.csv


In [42]:
import numpy as np
import csv

# 假設 B_inv 是你的 16x16 矩陣
# 例如：B_inv = np.array([[1, 2, 3, ...], [...], ...]) 

# 將矩陣存成CSV檔案
filename = 'matrix_g.csv'
np.savetxt(filename, g, delimiter=",", fmt="%.10e")

print(f"矩陣已成功存成CSV檔案：{filename}")

矩陣已成功存成CSV檔案：matrix_g.csv


In [21]:
#Build each set of circuits with Initialstates, cnot gate, measurements
#Cnot gate
Cnotgate = QuantumCircuit(qreg, creg)
Cnotgate.cx(0, 1)
Circuit = {}
for MeasName, Measure in Measurement.items():
    for InitName, Initial in InitialState.items():
        circuit_temp = q.circuit.QuantumCircuit.compose(Initial, Cnotgate)
        circuit = q.circuit.QuantumCircuit.compose(circuit_temp, Measure)
        Circuit.update({f'{InitName}_{MeasName}':circuit})

In [34]:
shots = 1000
sampler = Sampler(mode=backend)

job = {}
for Name, Cir in Circuit.items():
    CirTran = q.compiler.transpile(Cir, backend=backend, optimization_level=0)
    job.update({Name:sampler.run([CirTran], shots=shots)})

In [43]:
result_origin = []
for index, job_name in job.items():
    res = job_name.result()
    result_origin.append(res[0].data.c.get_counts())
    print(res)
result_origin

PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=1000, num_bits=2>)), metadata={'shots': 1000, 'circuit_metadata': {}})], metadata={'version': 2})
PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=1000, num_bits=2>)), metadata={'shots': 1000, 'circuit_metadata': {}})], metadata={'version': 2})
PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=1000, num_bits=2>)), metadata={'shots': 1000, 'circuit_metadata': {}})], metadata={'version': 2})
PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=1000, num_bits=2>)), metadata={'shots': 1000, 'circuit_metadata': {}})], metadata={'version': 2})
PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=1000, num_bits=2>)), metadata={'shots': 1000, 'circuit_metadata': {}})], metadata={'version': 2})
PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=1000, num_bits=2>)), metadata={'shot

[{'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'00': 1000},
 {'10': 508, '00': 492},
 {'10': 492, '00': 508},
 {'00': 1000},
 {'00': 503, '10': 497},
 {'10': 482, '00': 518},
 {'00': 517, '10': 483},
 {'00': 995, '10': 5},
 {'00': 496, '10': 504},
 {'10': 494, '00': 506},
 {'00': 510, '10': 490},
 {'00': 999, '10': 1},
 {'00': 514, '10': 486},
 {'00': 465, '10': 535},
 {'10': 511, '00': 489},
 {'00': 997, '10': 3},
 {'10': 538, '00': 462},
 {'00': 504, '10': 496},
 {'10': 517, '00': 483},
 {'00': 495, '10': 505},
 {'00': 999, '10': 1},
 {'10': 507, '00': 493},
 {'10': 498, '00': 502},
 {'00': 501, '10': 499},
 {'00': 996, '10': 4},
 {'10': 525, '00': 475},
 {'10': 505, '00': 495},
 {'00': 478, '10': 522},
 {'00': 999, '10': 1},
 {'00': 473, '10': 527},
 {'10': 488, '00': 512},
 {'10': 485, '00': 515},
 {'00':

In [37]:
# Required keys for each dictionary
required_keys = ['00', '01', '10', '11']

# Function to fill missing keys with 0
def fill_missing_keys(dicts, keys):
    return [{k: d.get(k, 0) for k in keys} for d in dicts]

# Fill missing keys in the input data
result = fill_missing_keys(result_origin, required_keys)

In [38]:
#Create GramMatrix g
g_cnot = np.ones((16,16))
for i in range (len(result)):
    row = int(i/16)
    column = i%16
    if i%16 == 0:
        if i/16 <= 3:
            g_cnot[row][column] = (result[i]['00'] - result[i]['10'] + result[i]['01'] - result[i]['01'])/1000
        else:
            g_cnot[row][column] = (result[i]['00'] + result[i]['10'] - result[i]['01'] - result[i]['01'])/1000
    else:
        g_cnot[row][column] = (result[i]['00'] - result[i]['10'] - result[i]['01'] + result[i]['01'])/1000
g_cnot

array([[ 1.   ,  1.   ,  1.   ,  1.   ,  1.   ,  1.   ,  1.   ,  1.   ,
         1.   ,  1.   ,  1.   ,  1.   ,  1.   ,  1.   ,  1.   ,  1.   ],
       [-0.002,  0.032,  0.942,  0.034, -0.018, -0.122,  0.934,  0.014,
         0.058, -0.036,  0.938,  0.022,  0.   , -0.004,  0.946,  0.046],
       [-0.012,  0.04 , -0.022,  0.944, -0.014, -0.012,  0.032, -0.918,
         0.03 ,  0.004,  0.016, -0.002, -0.016,  0.016, -0.004, -0.046],
       [ 0.94 , -0.926,  0.042, -0.02 , -0.944,  0.93 , -0.016, -0.016,
        -0.07 , -0.016, -0.016, -0.008,  0.058,  0.052, -0.014, -0.002],
       [-0.482,  0.515,  0.502,  0.502,  0.469,  0.51 ,  0.487,  0.497,
         0.508,  0.456,  0.974,  0.49 ,  0.514,  0.483,  0.523,  0.501],
       [-0.031, -0.012,  0.451,  0.015,  0.009, -0.01 ,  0.458, -0.003,
         0.473,  0.445,  0.917,  0.454, -0.011, -0.017,  0.452,  0.039],
       [ 0.054, -0.047, -0.009,  0.469,  0.019,  0.037, -0.036, -0.455,
        -0.037, -0.007, -0.033, -0.029,  0.47 , -0.444,  0

In [49]:
#Calculate Readout Matrix by the quation Uerror = B^-1 * g_cnot * A^-1
ErrorCnot = np.matmul(B_inv, np.matmul(g_cnot, A_inv))
ErrorCnot_inv = inv(U_cnot)
print(ErrorCnot)
print(ErrorCnot_inv)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-3.45716279e-02  1.01928509e+00  9.34920227e-02  4.39215919e-02
   1.97484639e-02 -4.06531463e-02 -4.04788492e-02 -9.41591658e-05
  -3.46329583e-02  4.14549023e-02 -1.58904788e-02 -2.99599965e-02
   3.02978896e-02 -9.74708733e-03 -4.24083579e-02 -4.39672718e-02]
 [-1.44613931e-02  4.02312892e-02  1.15167367e-02 -8.05011475e-02
   3.63411766e-02 -9.14332429e-03 -4.52239391e-02  7.80421849e-02
  -6.79773823e-02  2.77351187e-02 -3.35015758e-02 -1.60747605e-02
   3.26873799e-02 -4.68315641e-02  9.74674826e-01  1.22333056e-02]
 [-4.59552518e-04 -2.86514249e-02 -4.32175855e-02 -1.65039052e-02
   2.86542881e-02 -9.43080110e-03  5.43210028e-02  6.19167905e-02
   1.13117679e-01 -1.18710419e-01 -5.02576909e-02  3.01992974e-02
   1.21

In [48]:
#Construct Ideal Ucnot
pauli = []
pauli.append(Pauli("I").to_matrix())
pauli.append(Pauli("X").to_matrix())
pauli.append(Pauli("Y").to_matrix())
pauli.append(Pauli("Z").to_matrix())

Cnot = np.array([[1, 0, 0, 0],
                [0, 1, 0, 0],
                    [0 ,0, 0, 1],
                      [0, 0, 1, 0]])

pauli_pair = []
for i in pauli:
    for j in pauli:
        pauli_pair.append(np.kron(i, j))

IdealCnot = np.ones((16,16),  dtype=complex)
for i in range(len(pauli_pair)):
    for j in range(len(pauli_pair)):
        temp1 = np.matmul(pauli_pair[i], Cnot)
        temp2 = np.matmul(temp1, pauli_pair[j])
        IdealCnot[i][j] = np.trace(temp2)*(1/4)
print(IdealCnot)

[[ 0.5+0.j   0.5+0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0.5+0.j  -0.5+0.j
   0. +0.j   0. +0.j ]
 [ 0.5+0.j   0.5+0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j  -0.5+0.j   0.5+0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0.5+0.j   0. -0.5j  0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0.5+0.j   0. +0.5j]
 [ 0. +0.j   0. +0.j   0. +0.5j  0.5+0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. -0.5j  0.5+0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0.5+0.j   0.5+0.j   0. +0.j
   0. +0.j   0. -0.5j  0. +0.5j  0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0.5+0.j   0.5+0.j   0. +0.j
   0. +0.j   0. +0.5j  0. -0.5j  0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.j

In [51]:
#Construct Error channel
LambdaError = np.matmul(inv(IdealCnot), ErrorCnot)
print(LambdaError)

[[ 4.71503066e-01+0.j          4.40918525e-01+0.j
  -2.67153094e-02+0.j          2.42340373e-02+0.j
   3.99508324e-01+0.j         -3.24830893e-01+0.j
  -2.87043859e-01+0.j         -8.79921186e-02+0.j
   4.00163705e-01+0.j         -2.69059462e-01+0.j
   1.15225236e-03+0.j          3.07539423e-01+0.j
   5.11562695e-01+0.j         -5.12518524e-01+0.j
  -2.75484270e-02+0.j         -3.48308024e-02+0.j        ]
 [ 4.93925306e-01+0.j          5.78366564e-01+0.j
   1.20207332e-01+0.j          1.96875546e-02+0.j
  -3.79759860e-01+0.j          2.84177747e-01+0.j
   2.46565010e-01+0.j          8.78979594e-02+0.j
  -4.34796663e-01+0.j          3.10514365e-01+0.j
  -1.70427312e-02+0.j         -3.37499419e-01+0.j
  -4.81264805e-01+0.j          5.02771437e-01+0.j
  -1.48599309e-02+0.j         -9.13646942e-03+0.j        ]
 [-8.37133180e-03-0.00601698j  9.21693601e-02+0.02115748j
   6.05137033e-01+0.0258162j   2.61542816e-02+0.50526626j
  -3.69007413e-01+0.01769201j  2.50201292e-01-0.00605817j
   2.893

In [ ]:
# P = np.array([[1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],\
#               [1,  1, -1, -1,  1,  1, -1, -1,  1,  1, -1, -1,  1,  1, -1, -1],\
#               [1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1],\
#               [1, -1, -1,  1,  1, -1, -1,  1,  1, -1, -1,  1,  1, -1, -1,  1],\
#               [1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1],\
#               [1,  1, -1, -1,  1,  1, -1, -1, -1, -1,  1,  1, -1, -1,  1,  1],\
#               [1, -1,  1, -1,  1, -1,  1, -1, -1,  1, -1,  1, -1,  1, -1,  1],\
#               [1, -1, -1,  1,  1, -1, -1,  1, -1,  1,  1, -1, -1,  1,  1, -1],\
#               [1,  1,  1,  1, -1, -1, -1, -1,  1,  1,  1,  1, -1, -1, -1, -1],\
#               [1,  1, -1, -1, -1, -1,  1,  1,  1,  1, -1, -1, -1, -1,  1,  1],\
#               [1, -1,  1, -1, -1,  1, -1,  1,  1, -1,  1, -1, -1,  1, -1,  1],\
#               [1, -1, -1,  1, -1,  1,  1, -1,  1, -1, -1,  1, -1,  1,  1, -1],\
#               [1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1],\
#               [1,  1, -1, -1, -1, -1,  1,  1, -1, -1,  1,  1,  1,  1, -1, -1],\
#               [1, -1,  1, -1, -1,  1, -1,  1, -1,  1, -1,  1,  1, -1,  1, -1],\
#               [1, -1, -1,  1, -1,  1,  1, -1, -1,  1,  1, -1,  1, -1, -1,  1]])